In [ ]:
!nvidia-smi -L

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

### Some basic setup, import:

In [ ]:
# %matplotlib inline

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import pandas as pd
import numpy as np
import os, json, cv2, random
import glob
# from matplotlib import pyplot as plt
from IPython import display
from IPython.display import Image
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model

import IPython

Set seed for reproducibility

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(seed=SEED)
torch.manual_seed(SEED)

In [ ]:
def imshow(img):
    _, ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

# Run a pre-trained detectron2 model

In [ ]:
# !wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
imshow(im)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
imshow(out.get_image()[:, :, ::-1])

# Train on a custom dataset


## Prepare the dataset

In [ ]:
# dataset_name = "my_dataset_train"
# metadata = {}
# json_file = "path_to_.json"
# image_root = "path_to_image_root"

In [ ]:
dataset_name = "glass3"
metadata = {}
json_file = "./data/glass1/result.json"
image_root = "./data/glass1"

In [ ]:
val_dataset_name = "val_glass3"
val_metadata = {}
val_json_file = "./data/valid20/result.json"
val_image_root = "./data/valid20"

In [ ]:
register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
register_coco_instances(val_dataset_name, val_metadata, val_json_file, val_image_root)
val_metadata = MetadataCatalog.get(val_dataset_name)
val_dataset_dicts = DatasetCatalog.get(val_dataset_name)

In [ ]:
def random_sample(dataset_dicts, number, metadata):
    for d in random.sample(dataset_dicts, number):
        img = cv2.imread(d["file_name"])
        visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
        vis = visualizer.draw_dataset_dict(d)
        imshow(vis.get_image()[:, :, ::-1])

In [ ]:
random_sample(dataset_dicts, 5, metadata)

## Train

Now, let's fine-tune a COCO-pretrained R50-FPN Mask R-CNN model on the dataset.


In [ ]:
def train(dataset_name, 
          images_per_batch=2, 
          learning_rate=0.00025, 
          iteration=300, 
          batch_size_per_image=512, 
          num_classes=1, 
          test=False, 
          test_dataset_name="",
          init_model="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",
          retinaNet=False):
    
    cfg = get_cfg()

    # cfg.MODEL.DEVICE='cpu'

    cfg.merge_from_file(model_zoo.get_config_file(init_model))
    cfg.DATASETS.TRAIN = (dataset_name,)

    if test:
        cfg.DATASETS.TEST = (test_dataset_name,)
    else:
        cfg.DATASETS.TEST = ()

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(init_model)  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = images_per_batch  # This is the real "batch size" commonly known to deep learning people (number of images per batch)
    cfg.SOLVER.BASE_LR = learning_rate  
    cfg.SOLVER.MAX_ITER = iteration
    cfg.SOLVER.STEPS = []        # do not decay learning rate
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = batch_size_per_image   # The "RoIHead batch size". 128 is faster, (default: 512). Number of regions per image used to train RPN.
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes  
    
    if retinaNet:
        cfg.MODEL.RETINANET.NUM_CLASSES = num_classes
    
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg) 
    trainer.resume_or_load(resume=False)
    trainer.train()

    if test:
        cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
        predictor = DefaultPredictor(cfg)
        evaluator = COCOEvaluator(test_dataset_name, output_dir="./output")
        test_result = trainer.test(cfg, predictor.model, evaluator)
        return cfg, test_result

    return cfg, ""

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=1, 
                         test=True, 
                         test_dataset_name=val_dataset_name)

In [ ]:
df = pd.DataFrame(test_result, columns=test_result.keys())

In [ ]:
df

## Inference & evaluation using the trained model
Let's run inference with the trained model on the validation dataset. First, let's create a predictor using the model we just trained:



In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluator = COCOEvaluator("val_glass3", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "val_glass3")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

In [ ]:
def evaluate(image, metadata, instance_mode=False):
    im = cv2.imread(image)
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    
    v = {}
    if instance_mode:
        v = Visualizer(im[:, :, ::-1],
                        metadata=metadata, 
                        scale=0.5, 
                        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
    else:
        v = Visualizer(im[:, :, ::-1],
                        metadata=metadata, 
                        scale=0.5,
        )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    imshow(out.get_image()[:, :, ::-1])

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass451.jpg", metadata, instance_mode=True)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass451.jpg", metadata, instance_mode=False)

In [ ]:
def random_test_sample(dataset_dicts, number, metadata, instance_mode=False):
    for i in range(number):
        images = glob.glob(dataset_dicts + "*.jpg")
        random_image = random.choice(images)
        evaluate(random_image, metadata, instance_mode)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/"
random_test_sample(test_dataset_path, 5, metadata)

## Automation of training, parameter setting

In [ ]:
# images_per_batch = [2, 4, 8, 16, 32, 64]
# learning_rate = [0.01, 0.0025, 0.001, 0.00025, 0.0001, 0.000025, 0.00001]
# batch_size_per_image = [128, 256, 512] 
# iteration = 300
# num_classes = 4

In [ ]:
# result = []
# for ipb in images_per_batch:
#     for lr in learning_rate:
#         for bspi in batch_size_per_image:
#             cfg, test_result = train(dataset_name, 
#                                      images_per_batch=ipb, 
#                                      learning_rate=lr, 
#                                      iteration=iteration, 
#                                      batch_size_per_image=bspi, 
#                                      num_classes=num_classes, 
#                                      test=True, 
#                                      test_dataset_name=val_dataset_name)

#             result.append([ipb, lr, bspi, cfg, test_result])

In [ ]:
# df = pd.DataFrame()
# for i in range(len(result)):
#     temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
#     temp = temp.T

#     images_per_batch = pd.Series([result[i][0], result[i][0]], index=[0, 1])
#     learning_rate = pd.Series([result[i][1], result[i][1]], index=[0, 1])
#     batch_size_per_image = pd.Series([result[i][2], result[i][2]], index=[0, 1])

#     temp['images_per_batch'] = images_per_batch.values
#     temp['learning_rate'] = learning_rate.values
#     temp['batch_size_per_image'] = batch_size_per_image.values

#     df = pd.concat([df, temp])

In [ ]:
# df

## Train with more data

### Train2

In [ ]:
dataset_name = "glassAndMetal2"
metadata = {}
json_file = "/content/drive/MyDrive/Detectron/data/glassAndMetal2/result.json"
image_root = "/content/drive/MyDrive/Detectron/data/glassAndMetal2"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=2, 
                         test=False, 
                         test_dataset_name=val_dataset_name)

### Evaluate2

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass460.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/metal400.jpg", metadata, instance_mode=False)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/"
random_test_sample(test_dataset_path, 3, metadata)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/"
random_test_sample(test_dataset_path, 3, metadata)

### Train3

In [ ]:
dataset_name = "glassAndMetal4"
metadata = {}
json_file = "/content/drive/MyDrive/Detectron/data/glassAndMetal4/result.json"
image_root = "/content/drive/MyDrive/Detectron/data/glassAndMetal4"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=2, 
                         test=False, 
                         test_dataset_name=val_dataset_name)

### Evaluate3

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass460.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/metal400.jpg", metadata, instance_mode=False)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/"
random_test_sample(test_dataset_path, 3, metadata)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/"
random_test_sample(test_dataset_path, 3, metadata)

### Train4

In [ ]:
dataset_name = "glassAndMetal7"
metadata = {}
json_file = "/content/drive/MyDrive/Detectron/data/glassAndMetal7/result.json"
image_root = "/content/drive/MyDrive/Detectron/data/glassAndMetal7"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=2, 
                         test=False, 
                         test_dataset_name=val_dataset_name)

### Evaluate4

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass460.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/metal400.jpg", metadata, instance_mode=False)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/"
random_test_sample(test_dataset_path, 3, metadata)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/"
random_test_sample(test_dataset_path, 3, metadata)

### Train5

In [ ]:
dataset_name = "glassAndMetalAndPlastic8"
metadata = {}
json_file = "/content/drive/MyDrive/Detectron/data/glassAndMetalAndPlastic8/result.json"
image_root = "/content/drive/MyDrive/Detectron/data/glassAndMetalAndPlastic8"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=3, 
                         test=False, 
                         test_dataset_name=val_dataset_name)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

### Evaluate5

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass460.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/metal400.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/plastic/test/plastic433.jpg", metadata, instance_mode=False)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/"
random_test_sample(test_dataset_path, 3, metadata)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/"
random_test_sample(test_dataset_path, 3, metadata)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/plastic/test/"
random_test_sample(test_dataset_path, 3, metadata)

### Train6

In [ ]:
dataset_name = "glassAndMetalAndPlasticAndPaper9"
metadata = {}
json_file = "/content/drive/MyDrive/Detectron/data/glassAndMetalAndPlasticAndPaper9/result.json"
image_root = "/content/drive/MyDrive/Detectron/data/glassAndMetalAndPlasticAndPaper9"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=4, 
                         test=True, 
                         test_dataset_name=val_dataset_name)

### Evaluate6

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass460.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/metal400.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/plastic/test/plastic433.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/paper/test/paper584.jpg", metadata, instance_mode=False)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

### Train7

In [ ]:
dataset_name = "glassAndMetalAndPlasticAndPaper400"
metadata = {}
json_file = "/content/drive/MyDrive/Detectron/data/glassMetalPaperPlastic400/result.json"
image_root = "/content/drive/MyDrive/Detectron/data/glassMetalPaperPlastic400"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
cfg, test_result = train(dataset_name, 
                         images_per_batch=2, 
                         learning_rate=0.00025, 
                         iteration=3000, 
                         batch_size_per_image=128, 
                         num_classes=4, 
                         test=True, 
                         test_dataset_name=val_dataset_name)

### Evaluate7

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/glass/test/glass460.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/metal/test/metal400.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/plastic/test/plastic433.jpg", metadata, instance_mode=False)

In [ ]:
evaluate("/content/drive/MyDrive/Detectron/data/dataset-resized/paper/test/paper584.jpg", metadata, instance_mode=False)

In [ ]:
test_dataset_path = "/content/drive/MyDrive/Detectron/data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

## Automation of training, parameter setting

In [ ]:
def get_best_result(df, number=3, segmentation=True):
    dfbbox = df.loc[df['Result-type'] == "bbox"].sort_values(by='AP', ascending=False).head(number)
    if segmentation:
        dfsegm = df.loc[df['Result-type'] == "segm"].sort_values(by='AP', ascending=False).head(number)
        return dfbbox, dfsegm
    else:
        return dfbbox

In [ ]:
dataset_name = "glassAndMetalAndPlasticAndPaper400"
metadata = {}
json_file = "./data/glassMetalPaperPlastic400_2/result.json"
image_root = "./data/glassMetalPaperPlastic400_2"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
val_dataset_name = "valid20"
val_metadata = {}
val_json_file = "./data/valid20/result.json"
val_image_root = "./data/valid20"

register_coco_instances(val_dataset_name, val_metadata, val_json_file, val_image_root)
val_metadata = MetadataCatalog.get(val_dataset_name)
val_dataset_dicts = DatasetCatalog.get(val_dataset_name)

In [ ]:
images_per_batch = [2, 4, 8, 16, 32, 64]
learning_rate = [0.01, 0.0025, 0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 1500
num_classes = 4

In [ ]:
def save_result(result, model_name, segmentation=True):
    df = pd.DataFrame()
    for i in range(len(result)):
        temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
        temp = temp.T

        if segmentation:
            images_per_batch = pd.Series([result[i][0], result[i][0]], index=[0, 1])
            learning_rate = pd.Series([result[i][1], result[i][1]], index=[0, 1])
            batch_size_per_image = pd.Series([result[i][2], result[i][2]], index=[0, 1])
        else:
            images_per_batch = pd.Series([result[i][0]], index=[0])
            learning_rate = pd.Series([result[i][1]], index=[0])
            batch_size_per_image = pd.Series([result[i][2]], index=[0])

        temp['images_per_batch'] = images_per_batch.values
        temp['learning_rate'] = learning_rate.values
        temp['batch_size_per_image'] = batch_size_per_image.values

        df = pd.concat([df, temp])
        df.to_csv('result/' + str(model_name) + '.csv', index=True, index_label="Result-type") 

### Instance Segmentation

In [ ]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# t = torch.cuda.get_device_properties(0).total_memory
# r = torch.cuda.memory_reserved(0)
# a = torch.cuda.memory_allocated(0)
# f = r-a  # free inside reserved

In [ ]:
# f

#### COCO-pretrained R50-FPN Mask R-CNN

In [ ]:
# images_per_batch = [64, 32, 16, 8, 4, 2]
images_per_batch = [16, 8, 4, 2]
learning_rate = [0.01, 0.0025, 0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 100
num_classes = 4

In [ ]:
init_model="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

In [ ]:
result = []
for ipb in images_per_batch:
    for lr in learning_rate:
        for bspi in batch_size_per_image:
            cfg, test_result = train(dataset_name, 
                                     images_per_batch=ipb, 
                                     learning_rate=lr, 
                                     iteration=iteration, 
                                     batch_size_per_image=bspi, 
                                     num_classes=num_classes, 
                                     test=True, 
                                     test_dataset_name=val_dataset_name,
                                     init_model=init_model)

            result.append([ipb, lr, bspi, cfg, test_result])
            save_result(result=result, model_name="mask_rcnn_R_50_FPN_3x", segmentation=True)

In [ ]:
df = pd.DataFrame()
for i in range(len(result)):
    temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
    temp = temp.T

    images_per_batch = pd.Series([result[i][0], result[i][0]], index=[0, 1])
    learning_rate = pd.Series([result[i][1], result[i][1]], index=[0, 1])
    batch_size_per_image = pd.Series([result[i][2], result[i][2]], index=[0, 1])

    temp['images_per_batch'] = images_per_batch.values
    temp['learning_rate'] = learning_rate.values
    temp['batch_size_per_image'] = batch_size_per_image.values

    df = pd.concat([df, temp])

In [ ]:
df

In [ ]:
df.to_csv('result2/mask_rcnn_R_50_FPN_3x.csv', index=True, index_label="Result-type") 

In [ ]:
df2 = pd.read_csv('result/mask_rcnn_R_50_FPN_3x.csv')

In [ ]:
dfbbox, dfsegm = get_best_result(df2, 5)

In [ ]:
dfbbox

In [ ]:
dfsegm

#### COCO-pretrained R101-FPN Mask R-CNN

In [ ]:
images_per_batch = [2, 4, 8, 16, 32, 64]
learning_rate = [0.01, 0.0025, 0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 1500
num_classes = 4

In [ ]:
init_model="COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"

In [ ]:
result = []
for ipb in images_per_batch:
    for lr in learning_rate:
        for bspi in batch_size_per_image:
            cfg, test_result = train(dataset_name, 
                                     images_per_batch=ipb, 
                                     learning_rate=lr, 
                                     iteration=iteration, 
                                     batch_size_per_image=bspi, 
                                     num_classes=num_classes, 
                                     test=True, 
                                     test_dataset_name=val_dataset_name,
                                     init_model=init_model)

            result.append([ipb, lr, bspi, cfg, test_result])
            save_result(result=result, model_name="mask_rcnn_R_101_FPN_3x", segmentation=True)

In [ ]:
df = pd.DataFrame()
for i in range(len(result)):
    temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
    temp = temp.T

    images_per_batch = pd.Series([result[i][0], result[i][0]], index=[0, 1])
    learning_rate = pd.Series([result[i][1], result[i][1]], index=[0, 1])
    batch_size_per_image = pd.Series([result[i][2], result[i][2]], index=[0, 1])

    temp['images_per_batch'] = images_per_batch.values
    temp['learning_rate'] = learning_rate.values
    temp['batch_size_per_image'] = batch_size_per_image.values

    df = pd.concat([df, temp])

In [ ]:
df

In [ ]:
df.to_csv('result2/mask_rcnn_R_101_FPN_3x.csv', index=True, index_label="Result-type") 

In [ ]:
df2 = pd.read_csv('result/mask_rcnn_R_101_FPN_3x.csv')

In [ ]:
dfbbox, dfsegm = get_best_result(df2)

In [ ]:
dfbbox

In [ ]:
dfsegm

### Detection

#### COCO-pretrained R50-FPN Faster R-CNN

In [ ]:
images_per_batch = [2, 4, 8, 16, 32, 64]
learning_rate = [0.01, 0.0025, 0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 1500
num_classes = 4

In [ ]:
init_model="COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [ ]:
result = []
for ipb in images_per_batch:
    for lr in learning_rate:
        for bspi in batch_size_per_image:
            cfg, test_result = train(dataset_name, 
                                     images_per_batch=ipb, 
                                     learning_rate=lr, 
                                     iteration=iteration, 
                                     batch_size_per_image=bspi, 
                                     num_classes=num_classes, 
                                     test=True, 
                                     test_dataset_name=val_dataset_name,
                                     init_model=init_model)

            result.append([ipb, lr, bspi, cfg, test_result])
            save_result(result=result, model_name="faster_rcnn_R_50_FPN_3x", segmentation=False)

In [ ]:
df = pd.DataFrame()
for i in range(len(result)):
    temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
    temp = temp.T

    images_per_batch = pd.Series([result[i][0]], index=[0])
    learning_rate = pd.Series([result[i][1]], index=[0])
    batch_size_per_image = pd.Series([result[i][2]], index=[0])

    temp['images_per_batch'] = images_per_batch.values
    temp['learning_rate'] = learning_rate.values
    temp['batch_size_per_image'] = batch_size_per_image.values

    df = pd.concat([df, temp])

In [ ]:
df

In [ ]:
df.to_csv('result2/faster_rcnn_R_50_FPN_3x.csv', index=True, index_label="Result-type") 

In [ ]:
df2 = pd.read_csv('result/faster_rcnn_R_50_FPN_3x.csv')

In [ ]:
dfbbox = get_best_result(df2, 5, segmentation=False)

In [ ]:
dfbbox

#### COCO-pretrained R101-FPN Faster R-CNN

In [ ]:
images_per_batch = [2, 4, 8, 16, 32, 64]
learning_rate = [0.01, 0.0025, 0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 1500
num_classes = 4

In [ ]:
init_model="COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"

In [ ]:
result = []
for ipb in images_per_batch:
    for lr in learning_rate:
        for bspi in batch_size_per_image:
            cfg, test_result = train(dataset_name, 
                                     images_per_batch=ipb, 
                                     learning_rate=lr, 
                                     iteration=iteration, 
                                     batch_size_per_image=bspi, 
                                     num_classes=num_classes, 
                                     test=True, 
                                     test_dataset_name=val_dataset_name,
                                     init_model=init_model)

            result.append([ipb, lr, bspi, cfg, test_result])
            save_result(result=result, model_name="faster_rcnn_R_101_FPN_3x", segmentation=False)

In [ ]:
df = pd.DataFrame()
for i in range(len(result)):
    temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
    temp = temp.T

    images_per_batch = pd.Series([result[i][0]], index=[0])
    learning_rate = pd.Series([result[i][1]], index=[0])
    batch_size_per_image = pd.Series([result[i][2]], index=[0])

    temp['images_per_batch'] = images_per_batch.values
    temp['learning_rate'] = learning_rate.values
    temp['batch_size_per_image'] = batch_size_per_image.values

    df = pd.concat([df, temp])

In [ ]:
df

In [ ]:
df.to_csv('result2/faster_rcnn_R_101_FPN_3x.csv', index=True, index_label="Result-type") 

In [ ]:
df2 = pd.read_csv('result/faster_rcnn_R_101_FPN_3x.csv')

In [ ]:
dfbbox = get_best_result(df2, segmentation=False)

In [ ]:
dfbbox

#### COCO-pretrained R50-FPN RetinaNet

In [ ]:
images_per_batch = [2, 4, 8, 16, 32, 64]
learning_rate = [0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 1500
num_classes = 4

In [ ]:
init_model="COCO-Detection/retinanet_R_50_FPN_3x.yaml"

In [ ]:
result = []
for ipb in images_per_batch:
    for lr in learning_rate:
        for bspi in batch_size_per_image:
            cfg, test_result = train(dataset_name, 
                                     images_per_batch=ipb, 
                                     learning_rate=lr, 
                                     iteration=iteration, 
                                     batch_size_per_image=bspi, 
                                     num_classes=num_classes, 
                                     test=True, 
                                     test_dataset_name=val_dataset_name,
                                     init_model=init_model,
                                     retinaNet=True)

            result.append([ipb, lr, bspi, cfg, test_result])
            save_result(result=result, model_name="retinanet_R_50_FPN_3x", segmentation=False)

In [ ]:
df = pd.DataFrame()
for i in range(len(result)):
    temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
    temp = temp.T

    images_per_batch = pd.Series([result[i][0]], index=[0])
    learning_rate = pd.Series([result[i][1]], index=[0])
    batch_size_per_image = pd.Series([result[i][2]], index=[0])

    temp['images_per_batch'] = images_per_batch.values
    temp['learning_rate'] = learning_rate.values
    temp['batch_size_per_image'] = batch_size_per_image.values

    df = pd.concat([df, temp])

In [ ]:
df

In [ ]:
df.to_csv('result2/retinanet_R_50_FPN_3x.csv', index=True, index_label="Result-type") 

In [ ]:
df2 = pd.read_csv('result/retinanet_R_50_FPN_3x.csv')

In [ ]:
dfbbox = get_best_result(df2, 5, segmentation=False)

In [ ]:
dfbbox

#### COCO-pretrained R101-FPN RetinaNet

In [ ]:
images_per_batch = [2, 4, 8, 16, 32, 64]
learning_rate = [0.001, 0.00025, 0.0001, 0.000025, 0.00001]
batch_size_per_image = [128, 256, 512] 
iteration = 1500
num_classes = 4

In [ ]:
init_model="COCO-Detection/retinanet_R_101_FPN_3x.yaml"

In [ ]:
result = []
for ipb in images_per_batch:
    for lr in learning_rate:
        for bspi in batch_size_per_image:
            cfg, test_result = train(dataset_name, 
                                     images_per_batch=ipb, 
                                     learning_rate=lr, 
                                     iteration=iteration, 
                                     batch_size_per_image=bspi, 
                                     num_classes=num_classes, 
                                     test=True, 
                                     test_dataset_name=val_dataset_name,
                                     init_model=init_model,
                                     retinaNet=True)

            result.append([ipb, lr, bspi, cfg, test_result])
            save_result(result=result, model_name="retinanet_R_101_FPN_3x", segmentation=False)

In [ ]:
df = pd.DataFrame()
for i in range(len(result)):
    temp = pd.DataFrame(result[i][4], columns=result[i][4].keys())
    temp = temp.T

    images_per_batch = pd.Series([result[i][0]], index=[0])
    learning_rate = pd.Series([result[i][1]], index=[0])
    batch_size_per_image = pd.Series([result[i][2]], index=[0])

    temp['images_per_batch'] = images_per_batch.values
    temp['learning_rate'] = learning_rate.values
    temp['batch_size_per_image'] = batch_size_per_image.values

    df = pd.concat([df, temp])

In [ ]:
df

In [ ]:
df.to_csv('result2/retinanet_R_101_FPN_3x.csv', index=True, index_label="Result-type") 

In [ ]:
df2 = pd.read_csv('result/retinanet_R_101_FPN_3x.csv')

In [ ]:
dfbbox = get_best_result(df2, segmentation=False)

In [ ]:
dfbbox

## Best parameters

In [ ]:
dataset_name = "glassMetalPapperPlastic"
metadata = {}
json_file = "./data/glassMetalPapperPlastic/result.json"
image_root = "./data/glassMetalPapperPlastic"

register_coco_instances(dataset_name, metadata, json_file, image_root)
metadata = MetadataCatalog.get(dataset_name)
dataset_dicts = DatasetCatalog.get(dataset_name)

In [ ]:
val_dataset_name = "valid25"
val_metadata = {}
val_json_file = "./data/valid25/result.json"
val_image_root = "./data/valid25"

register_coco_instances(val_dataset_name, val_metadata, val_json_file, val_image_root)
val_metadata = MetadataCatalog.get(val_dataset_name)
val_dataset_dicts = DatasetCatalog.get(val_dataset_name)

#### NO pretrained model

In [ ]:
# init_model="Misc/scratch_mask_rcnn_R_50_FPN_9x_gn.yaml"

#### COCO-pretrained R50-FPN Mask R-CNN

In [ ]:
images_per_batch = 8
learning_rate = 0.01
batch_size_per_image = 512 
iteration = 300
num_classes = 4

In [ ]:
init_model="COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"

In [ ]:
cfg, test_result = train(dataset_name,
                         images_per_batch=images_per_batch, 
                         learning_rate=learning_rate, 
                         iteration=iteration, 
                         batch_size_per_image=batch_size_per_image,
                         num_classes=num_classes, 
                         test=True, 
                         test_dataset_name=val_dataset_name,
                         init_model=init_model,
                         retinaNet=False)

In [ ]:
print(cfg.dump())

In [ ]:
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model, save_dir="output")
checkpointer.save("model_mask_rcnn_R_50_FPN_3x")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
print(cfg.dump())

In [ ]:
with open("output.yaml", "w") as f:
    f.write(cfg.dump())

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_mask_rcnn_R_50_FPN_3x.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
os.rename("./output","./output_mask_rcnn_R_50_FPN_3x")

#### COCO-pretrained R50-FPN Faster R-CNN

In [ ]:
images_per_batch = 16
learning_rate = 0.01
batch_size_per_image = 128 
iteration = 10000
num_classes = 4

In [ ]:
init_model="COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

In [ ]:
cfg, test_result = train(dataset_name,
                         images_per_batch=images_per_batch, 
                         learning_rate=learning_rate, 
                         iteration=iteration, 
                         batch_size_per_image=batch_size_per_image,
                         num_classes=num_classes, 
                         test=True, 
                         test_dataset_name=val_dataset_name,
                         init_model=init_model,
                         retinaNet=False)

In [ ]:
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model, save_dir="output")
checkpointer.save("model_faster_rcnn_R_50_FPN_3x")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_faster_rcnn_R_50_FPN_3x.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
os.rename("./output","./output_faster_rcnn_R_50_FPN_3x")

#### COCO-pretrained R50-FPN RetinaNet

In [ ]:
images_per_batch = 4
learning_rate = 0.001
batch_size_per_image = 256 
iteration = 10000
num_classes = 4

In [ ]:
init_model="COCO-Detection/retinanet_R_50_FPN_3x.yaml"

In [ ]:
cfg, test_result = train(dataset_name,
                         images_per_batch=images_per_batch, 
                         learning_rate=learning_rate, 
                         iteration=iteration, 
                         batch_size_per_image=batch_size_per_image,
                         num_classes=num_classes, 
                         test=True, 
                         test_dataset_name=val_dataset_name,
                         init_model=init_model,
                         retinaNet=True)

In [ ]:
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model, save_dir="output")
checkpointer.save("model_retinanet_R_50_FPN_3x")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_retinanet_R_50_FPN_3x.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_path = "./data/dataset-resized/test/"
random_test_sample(test_dataset_path, 10, metadata)

In [ ]:
os.rename("./output","./output_retinanet_R_50_FPN_3x")